In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import os
import matplotlib.pyplot as plt
import seaborn as sns

/Users/dam/miniforge3/envs/poemas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Limpiar memoria para Mac (MPS)
if torch.backends.mps.is_available():
    torch.mps.empty_cache()

device = torch.device("mps")
dtype = torch.float16

In [3]:
# Definición de rutas absolutas
BASE_MODEL = "/Users/dam/ESCOM/ML/Poemas/models/BASE_MODEL"
LORA_BASE_PATH = "/Users/dam/ESCOM/ML/Poemas/models/LoRa"

# Adaptadores LoRA con su ruta completa dentro de la carpeta LoRa
PATH_ALITERACION = os.path.join(LORA_BASE_PATH, "lora_aliteracion")
PATH_ANAFORA = os.path.join(LORA_BASE_PATH, "lora_anafora")
PATH_ANIMALIZACION = os.path.join(LORA_BASE_PATH, "lora_animalizacion")
PATH_ASINDENTON = os.path.join(LORA_BASE_PATH, "lora_asindeton")
PATH_COSIFICACION = os.path.join(LORA_BASE_PATH, "lora_cosificacion")
PATH_EPITETO = os.path.join(LORA_BASE_PATH, "lora_epiteto")
PATH_HIPERBOLE = os.path.join(LORA_BASE_PATH, "lora_hiperbole")
PATH_PARALELISMO = os.path.join(LORA_BASE_PATH, "lora_paralelismo")
PATH_PERSONIFICACION = os.path.join(LORA_BASE_PATH, "lora_personificacion")
PATH_PLEONASMO = os.path.join(LORA_BASE_PATH, "lora_pleonasmo")
PATH_POLISINDENTON = os.path.join(LORA_BASE_PATH, "lora_polisindeton")
PATH_SIMIL = os.path.join(LORA_BASE_PATH, "lora_simil")
PATH_METAFORA = os.path.join(LORA_BASE_PATH, "modelo_lora_final")


In [4]:
print(f"Cargando Tokenizer desde {BASE_MODEL}...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, local_files_only=True)

print(f"Cargando Modelo Base desde {BASE_MODEL} con MPS...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=dtype,
    device_map={"": "mps"},
    trust_remote_code=True,
    attn_implementation="eager",
    local_files_only=True
)

Cargando Tokenizer desde /Users/dam/ESCOM/ML/Poemas/models/BASE_MODEL...


`torch_dtype` is deprecated! Use `dtype` instead!


Cargando Modelo Base desde /Users/dam/ESCOM/ML/Poemas/models/BASE_MODEL con MPS...


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.25s/it]


In [5]:
print("Cargando Adaptadores LoRA...")
try:
    # Cargamos el primer adaptador para inicializar el PeftModel
    model = PeftModel.from_pretrained(
        model,
        PATH_METAFORA,
        adapter_name="metafora"
    )

    # Cargamos el resto de los adaptadores vinculando cada ruta a su nombre
    model.load_adapter(PATH_ALITERACION, adapter_name="aliteracion")
    model.load_adapter(PATH_ANAFORA, adapter_name="anafora")
    model.load_adapter(PATH_ANIMALIZACION, adapter_name="animalizacion")
    model.load_adapter(PATH_ASINDENTON, adapter_name="asindeton")
    model.load_adapter(PATH_COSIFICACION, adapter_name="cosificacion")
    model.load_adapter(PATH_EPITETO, adapter_name="epiteto")
    model.load_adapter(PATH_HIPERBOLE, adapter_name="hiperbole")
    model.load_adapter(PATH_PARALELISMO, adapter_name="paralelismo")
    model.load_adapter(PATH_PERSONIFICACION, adapter_name="personificacion")
    model.load_adapter(PATH_PLEONASMO, adapter_name="pleonasmo")
    model.load_adapter(PATH_POLISINDENTON, adapter_name="polisindeton")
    model.load_adapter(PATH_SIMIL, adapter_name="simil")

    model.eval()
    print(f"ADAPTADORES CARGADOS: {list(model.peft_config.keys())}")

except Exception as e:
    print(f"Error al cargar adaptadores: {e}")


Cargando Adaptadores LoRA...
ADAPTADORES CARGADOS: ['metafora', 'aliteracion', 'anafora', 'animalizacion', 'asindeton', 'cosificacion', 'epiteto', 'hiperbole', 'paralelismo', 'personificacion', 'pleonasmo', 'polisindeton', 'simil']


In [6]:
def generar_poema(palabra, figura="metafora"):
    model.set_adapter(figura)

    prompt = f'Escribe un poema usando la figura retórica "{figura}" con la palabra "{palabra}".\nPoema:\n'
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=80,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )

    resultado = tokenizer.decode(output[0], skip_special_tokens=True)
    poema = resultado.split("Poema:")[-1].strip()

    print(f"\n[{figura.upper()}] {palabra}:\n{poema}\n" + "-" * 30)


In [ ]:
def visualizar_atencion(palabra, figura="metafora", capa=-1, cabeza=0):
    model.set_adapter(figura)

    prompt = f'Escribe un poema usando la figura retórica "{figura}" con la palabra "{palabra}".\nPoema:\n'
    inputs_prompt = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output_tokens = model.generate(
            **inputs_prompt,
            max_new_tokens=40,
            do_sample=True,
            temperature=0.7
        )

        outputs = model(output_tokens, output_attentions=True)
        attentions = outputs.attentions

    attn_matrix = attentions[capa][0, cabeza].detach().cpu().float().numpy()
    full_tokens = [tokenizer.decode([t]) for t in output_tokens[0]]

    plt.figure(figsize=(14, 12))
    sns.heatmap(attn_matrix, xticklabels=full_tokens, yticklabels=full_tokens, cmap='magma')
    plt.title(f"Atención Completa: {figura.upper()} (Prompt + Generación)")
    plt.show()


In [10]:
if __name__ == "__main__":
    figuras_a_probar = [
        #"aliteracion",
        #"anafora",
        "animalizacion",
        #"asindeton",
        #"epiteto",
        #"hiperbole",
        #"paralelismo",
        #"pleonasmo",
        #"polisindeton",
        #"simil",
        #"metafora",
        "personificacion",
        "cosificacion"
    ]

    palabra_test = "Burócrata"

    for fig in figuras_a_probar:
        try:
            generar_poema(palabra_test, fig)
        except Exception as e:
            print(f"Error generando con {fig}: {e}")


[ANIMALIZACION] Burócrata:
la burócrata es un animal lento,
con garras de plomo y ojos de neón,
su paso es pesado, su mirada es dura,
en un mundo que se mueve con velocidad.
<|endoftext|>
------------------------------

[PERSONIFICACION] Burócrata:
la burócrata es una lista interminable,
de papeles y formularios sin fin,
de fechas y horas que se repiten,
en un ciclo sin descanso, sin fin.
<|endoftext|>
------------------------------

[COSIFICACION] Burócrata:
la burócrata es un peso que carga
sobre los hombros de la ciudad
un lastre que no se mueve
y que nos hace sentir cansados
<|endoftext|>
------------------------------
